# Warsztaty Python w Data Science

---
## Text Mining - część 1 z 3  

- ### Tokenizacja
- ### Statystyki
- ### Prosta Lemmatyzacja
- ### TF-IDF

https://github.com/MichalKorzycki/WarsztatPythonDataScience

https://drive.google.com/drive/folders/1HR8VCledCwD7BRMO1AucUM3x7cYC-AVT?usp=sharing

---
# Zadanie 1.
Wyciągnąć z _*kilku*_ ogłoszeń ich tytuły i treści

In [ ]:
import scrapy
import datetime
import pandas as pd
import scrapy.crawler as crawler
from bs4 import BeautifulSoup
from scrapy.exporters import CsvItemExporter
from scrapy.crawler import CrawlerProcess

url_results = []
desc_results = []
title_results = []


class GumtreeApartmentsSpider(scrapy.Spider):
    name = 'gumtreeapartmentsspider'
    start_urls = [ 
    
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/page-'+str(i)+'/v1c9073l3200001p'+str(i)  for i in range(2, 11)
        ]
    start_urls.append(
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1'
    )
    found_apartments = []
   
    custom_settings = {
        'DOWNLOAD_DELAY': '2.0',
        'ROBOTSTXT_OBEY': True,
        'AUTOTHROTTLE_ENABLED': True,
        'USER_AGENT': 'My Bot (email@myemail.com)'
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        titles = [flat.next_element for flat in soup.find_all('a', class_ = "href-link tile-title-text")] 
        links = ['https://www.gumtree.pl' + link.get('href')
                for link in soup.find_all('a', class_ ="href-link tile-title-text")]
            
        for item_url in links:
            yield scrapy.Request(item_url, self.parse_item)
        
    def parse_item(self, response): 
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        title = soup.find('span', class_ ="myAdTitle")
        description = soup.find('div', class_ ="description")
        item = {
            "url": response.url,
            "title": title,
            "description": description,
        }

        url_results.append(response.url)
        desc_results.append(description)
        title_results.append(title)
        
    def spider_closed(self, spider):
        spider.logger.info('Spider closed: %s', spider.name)
        
        df = pd.Dataframe({
            "title": title_results,
            "description": desc_results,
            "url": url_results
        })
        fname = f"gumtree-{now}.csv"
        print(fname)
        df.to_csv(fname)

In [ ]:
process = CrawlerProcess()
process.crawl(GumtreeApartmentsSpider)
process.start()

In [ ]:
df = pd.DataFrame({
            "title": title_results,
            "description": desc_results,
            "url": url_results
        })

In [ ]:
now = datetime.datetime.now().strftime("%Y-%m-%d")
fname = f"data\gumtree-{now}.csv"
print(fname)
df.to_csv(fname, sep="|")

---
# Tokenizacja

In [ ]:
import pandas as pd

data = pd.read_csv('data\gumtree-2021-03-09.csv', sep='|')
data.set_index('Unnamed: 0')

In [ ]:
opis = data['description'][0]
opis

In [ ]:
import re

def no_tags(s):
    return re.sub(r'<[^<]+?>','',s)

opis = no_tags(opis)
opis

In [ ]:
import re

tokenizer = re.compile(r'[^ąąćęńłóóśśżżź\w]+')
tokenized = tokenizer.split(opis)
str(tokenized)

In [ ]:
tokenized = [ x.lower() for x in tokenized ]
str(tokenized)

In [ ]:
def preprocessing(opis):
    opis = no_tags(opis)
    tokenized = tokenizer.split(opis)
    l = list(tokenized)
    l = [ x.lower() for x in l ]
    return l

In [ ]:
corpus=[]
n=4
for row in data.iterrows():
    opis = row[1][2]
    l = preprocessing(opis)
    corpus.append(l)
    n-=1
    if n==0: break

for opis in corpus:
    print(opis)
    print()

In [ ]:
---
# Statystyki

In [ ]:
corpus = []
for row in data.iterrows():
    opis = row[1][2]
    if type(opis) == str:
        l = preprocessing(opis)
        corpus.append(l)

    
print(f"Mamy tekstów: {len(corpus)}")

In [ ]:
all_words = []
for t in corpus:
    all_words += t
 
print(f"Mamy {len(all_words)} wyrazów")
all_words[:15]

In [ ]:
counter = {}

for w in all_words:
    counter[w] = counter.get(w,0)+1

print(f"Mamy {len(counter.keys())} RÓŻNYCH wyrazów")

In [ ]:
counted_words= [ (word,cnt) for word,cnt in counter.items() ]
counted_words[:4]

In [ ]:
from operator import itemgetter

counted_words.sort(key=itemgetter(1), reverse=True)
counted_words[:20]

In [ ]:
counted_words[-20:]

In [ ]:
counts = [ x[1] for x in counted_words ]
len(counts)

In [ ]:
sum(counts)

In [ ]:
sum(counts[:120])

In [ ]:
counted_words[110:130]

In [ ]:
count_df = pd.DataFrame(counts[:120])
count_df

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.dates as mdates

plt.figure(figsize=(24,12))
plt.style.use("dark_background")

chart = sns.scatterplot(
                     color='purple', 
                     data=count_df
                    )

---
# Prosta Lematyzacja

## _Lematyzacja_ - sprowadzenie wyrazu do formy podstawowej tak aby różne formy tego wyrazu (*kot*, *kota*, *kotu*) były rozpatrywane jako ten sam wyraz (*kot*) 

https://sjp.pl/
    
Słownik SJP.PL
Słownik języka polskiego, ortograficzny, wyrazów obcych i słownik do gier w jednym.

Słownik jest rozwijany z myślą o zastosowaniu do sprawdzania pisowni w programach open-source, do gier słownych (np. literaki) i do użytku online jako kilka rodzajów słowników w jednym.

Redakcją słownika zajmują się hobbyści.

Słownik jest udostępniany na otwartych licencjach (różnych w zależności od wersji).

In [ ]:
import gzip
import sys
import re

f = gzip.open('data/odm.txt.gz', 'rt', encoding='utf-8')
dictionary = {}

for x in f:
    t = x.strip().split(',')
    tt = [ x.strip().lower() for x in t]
    for w in tt[1:]: 
        dictionary[w]=tt[0]


In [ ]:
def lematize(w):
    return dictionary.get(w,w)

In [ ]:
corpusl = [[ lematize(x) for x in l ] for l in corpus]
for opis in corpusl[:4]:
    print(opis)
    print()

In [ ]:
all_words = []
for t in corpusl:
    all_words += t
 
print(f"Mamy {len(all_words)} wyrazów")
all_words[:15]

In [ ]:
counter = {}

for w in all_words:
    counter[w] = counter.get(w,0)+1

print(f"Mamy {len(counter.keys())} RÓŻNYCH wyrazów")

In [ ]:
from operator import itemgetter
counted_words= [ (word,cnt) for word,cnt in counter.items() ]
counted_words.sort(key=itemgetter(1), reverse=True)
counted_words[:20]

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.dates as mdates

counts = [ x[1] for x in counted_words ]
count_df = pd.DataFrame(counts[:120])
count_df

plt.figure(figsize=(24,12))
plt.style.use("dark_background")

chart = sns.scatterplot(
        color='purple', 
        data=count_df
        )

---
## Metryka TF-IDF
ile razy występuję wyraz *i* w tekście *j*
$${n}_{ij}$$ 
 ### Term Frequency (TF)
 
 $${tf}_{ij} = \frac{{n}_{ij}}{\sum{k}{{n}_{ik}}}$$
 
 W tekście *j* sprawdzamy ile proporcjonalnie do całości występuje wyraz *i*
### Inverted Document Frequency (IDF)

 $$idf_i = log \frac{|D|}{ \{ d: n_i \in d \}}$$
 
 licznik - liczba dokumentów
 
 mianownik - liczba dokumentów w którym wystapił wyraz *i*-ty 

ile razy występuję wyraz *i* w tekście *j*

$${n}_{ij}$$ 

 ### Term Frequency (TF)
 
 $${tf}_{ij} = \frac{{n}_{ij}}{\sum{k}{{n}_{ik}}}$$
 
 W tekście *j* sprawdzamy ile proporcjonalnie do całości występuje wyraz *i*

### Inverted Document Frequency (IDF)

 $$idf_i = log \frac{|D|}{ \{ d: n_i \in d \}}$$
 
 licznik - liczba dokumentów
 
 mianownik - liczba dokumentów w którym wystapił wyraz *i*-ty 

---
## Zadanie 1

Podać wyrazy z korpusu o największym `TF-IDF` z pominięciem wyrazów które wystepują _**raz**_.